In [8]:
from pathlib import Path
from datetime import date
import os
import load_csv
data_frame = load_csv.load("data/clean_data.csv")
context = load_csv.getContext()

AttributeError: module 'load_csv' has no attribute 'getContext'

In [2]:
data_frame.show()

+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|            order_id|     customer_id|    customer_name|product_id|        product_name|   product_category|    payment_type|qty| price|  datetime|             country|            city|ecommerce_website_name|  payment_txn_id|payment_txn_success|
+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|2NEBnZETy1FdFpKJf...|wEwuo1GDdnbC8ju5|   STEPHEN THOMAS|     14315|ESR Boost Kicksta...|tv, audio & cameras|             UPI|  3| 25.19|2022-05-19|       United States|       Blackhawk|       www.wayfair.com|XXFqXfizoOtsQ0S1|                  Y|
|cs4mesS4QiK

In [21]:
year_column = date_part(lit('YEAR'),'datetime').alias('year')
month_column = date_part(lit('month'),'datetime').alias('month')
day_column = date_part(lit('D'), 'datetime').alias('day')
data_frame = data_frame.withColumn('qty', col('qty').cast('integer'))
df_test = data_frame.select(data_frame.datetime, year_column, month_column, day_column, data_frame.country, data_frame.product_name, data_frame.qty)
# products = data_frame.select()
# df_test.groupBy('product_name').sum('qty').sort('country').show()
# df_test.groupBy('country').agg({'qty':'sum'}).show()#ps_sum(data_frame.qty)).show()
df_test.groupBy(['year', 'month','product_name']).agg({'qty':'sum'}).show()
df_test.agg(countDistinct('product_name')).show()
# df_test_2 = df_test.sort('datetime')
# df_test_2.show()
# df_test_2.agg()


+----+-----+--------------------+--------+
|year|month|        product_name|sum(qty)|
+----+-----+--------------------+--------+
|2022|    9|HAP Men's Modern ...|      10|
|2022|   12|Trendy Women's Co...|       6|
|2022|    1|GenX Mens Jazz Pr...|       3|
|2024|    5|OORA Men Running ...|       1|
|2022|    8|Walkaroo Mens Wf6...|      12|
|2023|    8|ZAPPER Silicone B...|      15|
|2023|    8|tZaro Genuine Lea...|       8|
|2022|    4|Aurelia Women's R...|       6|
|2022|    9|JBL Partybox 110 ...|       5|
|2022|    5|Bourge Mens Moda-...|       3|
|2022|    8|LUX COTT'S Wool M...|       8|
|2022|    3|Basilio Sliky Str...|       4|
|2023|    2|M Men Style Biker...|       8|
|2022|    7|FASHION VICTIM 10...|       7|
|2023|   11|Michelangelo Silv...|       8|
|2024|    5|Konvio Neer High ...|       2|
|2023|    2|Ridhi and Sidhi W...|       5|
|2022|    8|Aglance 7006 Flow...|      14|
|2023|   10|PC Jeweller The E...|      15|
|2023|   10|Brado Jewellery S...|       8|
+----+-----

In [61]:

print(os.path.abspath('') + "/test")

/home/miguel674-rev/2009_Big_Data/project-2/Revature_Project_2/data_analysis_and_cleaner/notebooks/test
